In [2]:
#llama using groq api

In [3]:
# First let's do an import
from dotenv import load_dotenv

In [4]:
from google.colab import userdata

# Get your API key from secrets
groq_api_key = userdata.get('GROQ_API_KEY')
print("Key loaded successfully:", groq_api_key[:5] + "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")  # Just to confirm, don’t print full key


Key loaded successfully: gsk_Zgsk_ELD2G4jD4fsyVPW43FTDWGdyb3FYAW7dzN7LytqYT3useaZbhUqm


In [5]:
# Next it's time to load the API keys into environment variables

load_dotenv(override=True)

False

In [6]:
# Check the Groq API key

import os
# Set API key manually
os.environ["GROQ_API_KEY"] = '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' # add your API key here: ######

groq_api_key = os.getenv('GROQ_API_KEY')

if groq_api_key:
    print(f"GROQ API Key exists and begins {groq_api_key[:8]}")
else:
    print("GROQ API Key not set")

GROQ API Key exists and begins gsk_ZHrr


In [7]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 6.1 MB/s eta 0:00:00


In [8]:
# And now - the all important import statement
# If you get an import error - head over to troubleshooting guide

from groq import Groq

In [9]:
# Create a Groq instance
groq = Groq()

In [9]:
# Create a list of messages in the familiar Groq format

messages = [{"role": "user", "content": "What is 2+2?"}]

In [10]:
# And now call it!

response = groq.chat.completions.create(model='llama-3.3-70b-versatile', messages=messages)
print(response.choices[0].message.content)


2 + 2 = 4.


In [11]:
# And now - let's ask for a question:

question = "Please propose a hard, challenging question to assess someone's IQ. Respond only with the question."
messages = [{"role": "user", "content": question}]


In [12]:
# ask it
response = groq.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=messages
)

question = response.choices[0].message.content

print(question)


A man is looking at a photograph of someone. His friend asks him, "Who is in the picture?" The man replies, "Brothers and sisters, I have none. But that man's father is my father's son." Who is in the picture?


In [14]:
#----------------------------------------------------------------------------------------------------

In [10]:
!pip install groq tavily-python faiss-cpu python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 33.1 MB/s eta 0:00:00


In [11]:
import os
import json
from typing import Dict, Any
from groq import Groq
from tavily import TavilyClient

In [12]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY", "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")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY", "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")

In [13]:
groq = Groq(api_key=os.environ["GROQ_API_KEY"])   # <-- use 'groq' instead of groq_client
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [14]:
# Tool Functions

def call_groq_llama(prompt: str, max_tokens: int = 256, temperature: float = 0.0) -> Dict[str, Any]:
    """Call Groq LLaMA using the official groq client."""
    try:
        response = groq.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": "You are a helpful controller agent."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=temperature,
        )
        text = response.choices[0].message.content
        return {"ok": True, "raw": response, "text": text}
    except Exception as e:
        return {"ok": False, "error": str(e), "raw": None}




In [15]:
def web_search_tool(query: str) -> str:
    try:
        results = tavily_client.search(query)
        if results and "results" in results:
            return "\n".join([r["content"] for r in results["results"][:3]])
        return "No search results."
    except Exception as e:
        return f"Web search failed: {e}"



In [16]:
def calculator_tool(expr: str) -> str:
    try:
        return str(eval(expr))
    except Exception as e:
        return f"Calculator error: {e}"

In [17]:
# =========================
# Controller Agent
# =========================
MASTER_PROMPT = """
You are a controller agent. A user will give you a query.
Decompose it into steps and decide which tool to call:
- "web_search" for internet lookup
- "calculator" for math
- Otherwise, answer directly.
Respond in JSON with fields: plan, tool, input.
Example: {"plan": "find weather", "tool": "web_search", "input": "weather in Paris"}
"""

def controller_agent(user_query: str) -> str:
    # Ask Groq to decide the plan
    planning_prompt = MASTER_PROMPT + f"\nUser query: {user_query}"
    resp = call_groq_llama(planning_prompt)
    if not resp["ok"]:
        return f"Error from Groq: {resp['error']}"

    try:
        parsed = json.loads(resp["text"])
    except Exception:
        return f"Could not parse Groq output: {resp['text']}"

    tool = parsed.get("tool", "none")
    tool_input = parsed.get("input", "")

    if tool == "web_search":
        tool_output = web_search_tool(tool_input)
    elif tool == "calculator":
        tool_output = calculator_tool(tool_input)
    else:
        tool_output = "Direct answer requested."

    final_prompt = f"User query: {user_query}\nTool used: {tool}\nTool input: {tool_input}\nTool output: {tool_output}\nProvide the final answer."
    final_resp = call_groq_llama(final_prompt)
    return final_resp["text"] if final_resp["ok"] else f"Final Groq error: {final_resp['error']}"




In [26]:
# Test Run------------------------
query = "What is the population of France divided by 2?"
answer = controller_agent(query)
print("User query:", query)
print("Agent answer:", answer)

User query: What is the population of France divided by 2?
Agent answer: The population of France is approximately 68.4 million people. To find the population divided by 2, we can perform the calculation:

68,400,000 / 2 = 34,200,000

So, the population of France divided by 2 is approximately 34.2 million.


In [1]:
#Document q&a(RAG)-----------------------------------------------

In [18]:
!pip install groq langchain faiss-cpu pypdf python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 15.2 MB/s eta 0:00:00


In [30]:
!pip install unstructured[docx] python-docx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 7.8 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=3e64fba63bf74d53044eaf47c77f420800ccab6ae2ffe045e6b11713f0caf5fe
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [33]:
!pip install langchain

In [31]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader


In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredWordDocumentLoader

In [24]:
# 📌 Document Ingestion

def load_and_chunk(file_path):
    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith(".docx"):
        loader = UnstructuredWordDocumentLoader(file_path)
    elif file_path.endswith(".txt"):
        loader = TextLoader(file_path)
    else:
        raise ValueError("Unsupported file format")

    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    return text_splitter.split_documents(docs)

In [25]:
# ============================
# 📌 API Key Setup
# ============================
os.environ["GROQ_API_KEY"] = "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"   # 🔑 Replace with your key
groq_client = Groq()









In [26]:
# ============================
# 📌 Document Ingestion
# ============================
def load_and_chunk(file_path):
    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith(".docx"):
        loader = UnstructuredWordDocumentLoader(file_path)
    elif file_path.endswith(".txt"):
        loader = TextLoader(file_path)
    else:
        raise ValueError("Unsupported file format")

    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    return text_splitter.split_documents(docs)

In [27]:
# ============================
# 📌 Build Vector DB
# ============================
def build_vectorstore(chunks):
    embeddings = OpenAIEmbeddings()  # If you prefer HuggingFace: HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return FAISS.from_documents(chunks, embeddings)

In [28]:
# ============================
# 📌 RAG-based Q&A Tool
# ============================
def doc_qa_tool(query: str, db) -> str:
    docs = db.similarity_search(query, k=3)
    context = "\n".join([d.page_content for d in docs])
    prompt = f"""You are answering based ONLY on the following context:
{context}

Question: {query}
Answer:"""

    resp = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )
    return resp.choices[0].message.content

In [37]:
# ============================
# 📌 Example Usage
# ============================
# Step 1: Upload a file in Colab first
from google.colab import files
uploaded = files.upload()

# Step 2: Load and chunk
file_path = list(uploaded.keys())[0]
chunks = load_and_chunk(file_path)

# Step 3: Build FAISS DB
db = build_vectorstore(chunks)

# Step 4: Ask a question
answer = doc_qa_tool("Summarize the key idea of this document.", db)
print("Q&A Answer:", answer)

Saving startup.pdf to startup.pdf


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [35]:
!pip install langchain-community

In [38]:
# --- Install dependencies ---
!pip install langchain faiss-cpu sentence-transformers pypdf python-docx docx2txt




In [39]:
# --- Imports ---
import os
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings



In [40]:
# --- Step 1: Load documents ---
def load_documents(file_path):
    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith(".txt"):
        loader = TextLoader(file_path)
    elif file_path.endswith(".docx"):
        loader = Docx2txtLoader(file_path)
    else:
        raise ValueError("Unsupported file type. Use PDF, TXT, or DOCX.")
    return loader.load()



In [41]:
# --- Step 2: Chunk text ---
def chunk_documents(docs):
    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
    return splitter.split_documents(docs)


In [42]:

# --- Step 3: Build FAISS vectorstore with HuggingFace embeddings ---
def build_vectorstore(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return FAISS.from_documents(chunks, embeddings)



In [43]:
# --- Step 4: Ask questions ---
def ask_question(db, query):
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    docs = retriever.get_relevant_documents(query)
    return docs



In [44]:
# --- Example usage ---
file_path = "/content/startup.pdf"  # upload your file to Colab first
docs = load_documents(file_path)
chunks = chunk_documents(docs)
db = build_vectorstore(chunks)

query = "What is the main conclusion of this document?"
results = ask_question(db, query)

print("\n🔎 Retrieved Answers:")
for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content[:300]}...")

/tmp/ipython-input-1608197478.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


🔎 Retrieved Answers:
1. TAM (Total Addressable Market) 
How many schools globally could use this? 
 
1.5 million K–12 schools globally 
 
Assume 40% have CCTV or could install it 
 
Estimated average annual spend for violence detection system: $500 per school/year 
 
TAM = 600,000 schools × $500 = $300 million/year 
 
 
SA...
2. around school settings. 
 
• In Pakistan, over 26 million students are enrolled at various educational levels, from pre-
primary to secondary education. They deserve safe and secure environment to nurture. 
 
• CCTVs are passive — they record but don’t detect or alert in real time, leaving violent 
...
3. • If we capture 1% of SAM in 3 years = 227 schools 
• 227 × $500 = $113,500/year revenue 
(With pilot programs and government partnerships, this could grow faster.)...


/tmp/ipython-input-1952118474.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


In [45]:
!pip install transformers accelerate bitsandbytes sentencepiece fastapi uvicorn nest_asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 10.0 MB/s eta 0:00:00


In [47]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Hugging Face repo with GSM8K finetune
model_name = "whoisltd/llama-3-2-gsm8k-finetuned"

# Load tokenizer safely
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=False,          # disable fast tokenizer (sometimes missing for custom models)
    trust_remote_code=True   # allow custom tokenizer logic
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

def ask_gsm8k(question, max_new_tokens=256):
    inputs = tokenizer(question, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 🔥 Test inference
print(ask_gsm8k("If there are 5 apples and you eat 2, how many are left?"))


TypeError: not a string